### Manacher's Algorithm
- 가장 긴 팰린드롬 부분문자열을 찾는 알고리즘
- 시간복잡도: $O(N)$ 

### Manacher이 사용하는 팰린드롬의 성질
- 어떤 문자열이 팰린드롬이면, 그 문자열의 중심으로부터 바깥쪽으로 한 문자씩 살펴봤을 때, 양 방향의 문자가 같은 문자이다.
  - "racecar"의 경우 중심 문자열이 e이고, 그 다음 양쪽으로 한문자씩 c, a, r이 같은 문자이다.
- 문자열을 순회하면서, 그동안 찾았던 팰린드롬과 관련된 정보를 유지해올 수 있다. 이미 찾은것보다 더 긴 회문을 만들 수 없는 문자열의 section들을 건너뛸 수 있는 잠재적인 가능성이 있다.
  - 탐색하고 있는 i 이전에 존재하면서 마지막 인덱스 j가 $j > i$ 인 팰린드롬을 갱신하는 방식으로 다음 인덱스에서도 팰린드롬을 찾을 수 있게 하고, 불필요한 계산을 줄인다.
- 문자열의 모든 문자는 팰린드롬의 중심이 될 수 있다. Manacher는 이 중심에서 확장하는 방식으로 팰린드롬을 찾는다. \
그리고 팰린드롬과 관련된 정보를 적절히 이용하기에 Naive한 $O(N^2)$ 방법보다 더 빠르다.
- 짝수 길이의 문자열을 대비하여 문자열의 양 끝과 각 문자 사이에 #을 넣어준다.
  - "racecar" -> "#r#a#c#e#c#a#r#"
  - 이를 통해 모든 팰린드롬이 하나의 중심을 가질 수 있게 된다.


### Manacher 동작
- 길이가 $N$ 인 문자열을 $S$, 이전 팰린드롬의 중심점을 $c$, 그 팰린드롬의 반지름을 $r$ 이라고 하고 $r = c = -1$ 으로 초기화한다.
- 길이가 $N$ 인 $man[]$ 배열을 초기화한다. 배열 안에는 각 문자 index를 중심으로 하는 팰린드롬의 반지름을 저장한다.
  1. 문자열의 처음과 끝, 그리고 각 문자 사이에 #을 넣어준다.
  2. $i = 0 \to N-1$ 까지 아래 과정을 반복한다.
  3. 기존에 구했던 man으로부터 자식 팰린드롬을 취한다.\
  $i \le r$ 에 대해 $man[i] := \min(r - i, man[c + (c-i)])$ 를 구하면 된다.
      - $r - i$ 는 자식 팰린드롬이 가질 수 있는 반지름의 최대 크기이고, $man[c + (c - i)]$ 는 건너편 자식 팰린드롬이 가지는 최대 반지름 값이 된다.
  4. 양쪽으로 포인터를 1칸씩 늘리면서 팰린드롬의 반지름을 갱신한다.
      - i로부터 $man[i]$ 개만큼 떨어진 문자열이 배열의 범위를 벗어나지 않으면서, 그 문자열의 양쪽 문자가 같다면 $man[i]++$ 를 한다.
  5. $i + man[i] > r$ 이 되면 $r$ 과 $c$ 를 갱신한다.
      - 이를 통해 다음 인덱스에서 빠르게 다음 팰린드롬을 찾아야 할 인덱스를 정할 수 있다.

### 구현

In [ ]:
def manacher(s, filler="#"):
  if len(s) < 2: return s

  _s = filler + filler.join(s) + filler
  p = [0] * len(_s)
  mx, loc = 0, 0
  index, maxlen = 0, 0
  for i in range(len(_s)):
    if i < mx and 2 * loc - i < len(_s):
      p[i] = min(mx - i, p[2 * loc - i])
    else:
      p[i] = 1

    while p[i] + i < len(_s) and i - p[i] >= 0 and _s[i - p[i]] == _s[i + p[i]]:
      p[i] += 1

    if i + p[i] > mx:
      mx = i + p[i]
      loc = i

    if p[i] > maxlen:
      index = i
      maxlen = p[i]
  s = _s[index - p[index] + 1:index + p[index]]
  return s.replace(filler, '')

### 개선된 구현
- r과 c 대신 왼쪽 포인터를 가르키는 L과 오른쪽 포인터를 가르키는 R으로 구현하고, R과 L을 동시에 늘리므로써 구현한 방법이다.
- 홀수길이의 팰린드롬 부분문자열과 짝수길이의 팰린드롬 부분문자열을 동시에 반환한다.
- z가 0이면 짝수 길이의 팰린드롬이고, 1이면 홀수길이의 팰린드롬이다. `len(s) & 1` 과 같다.
  - 0일경우 각 문자 사이에서 펠린드롬을 센다고 생각하면 된다. (ex. #a#b#c#), #에 대해서 팰린드롬을 세는 것
  - 따라서 p를 unpack할때는 even과 odd로 나누면 된다
- 아래 주석은 구한 p를 바탕으로 가장 긴 짝수 길이 팰린드롬 문자열과, 홀수길이 팰린드롬 문자열을 구한다.

In [ ]:
def LPS(s):
  n = len(s)
  p = [[0] * (n + 1), [0] * n]

  for z, p_z in enumerate(p):
    left, right = 0, 0
    for i in range(n):
      t = right - i + 1 - z
      if i < right:
        p_z[i] = min(t, p_z[left + t])
      L, R = i - p_z[i], i + p_z[i] - 1 + z
      while (L >= 1) and (R + 1 < n) and (s[L - 1] == s[R + 1]):
        p_z[i] += 1
        L -= 1 
        R += 1
      if R > right:
        left, right = L, R
  return p

  # i1, x1 = max(enumerate(p[0]), key=lambda x: x[1])
  # i2, x2 = max(enumerate(p[1]), key=lambda x: x[1])

  # return s[i1 - x1:i1 + x1], s[i2 - x2:i2 + x2 + 1]